In [1]:
#import the files 
import re 
import nltk
import ssl
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
cv = CountVectorizer()
train_data_2 = pd.read_csv('training_set.csv')
test_data = pd.read_csv('testing_set.csv')

X_train = cv.fit_transform(train_data_2["Review"]).toarray()
y_train = train_data_2["Liked"].values

X_test = cv.transform(test_data["Review"]).toarray()
y_test = test_data["Liked"].values

#train the model 
classifier = BernoulliNB(alpha=0.8)
classifier.fit(X_train, y_train)


# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,y_pred)

score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[114  21]
 [ 38 127]]


Accuracy is  80.33 %
Precision is  0.86
Recall is  0.77


In [3]:
len(y_pred)

300

In [4]:
business = pd.read_csv("cleaned_business.csv")
reviews = pd.read_csv("cleanish_reviews.csv") 
ids = pd.read_csv('list_buiss_id.txt', sep=" ", header=None)
ids.columns = ["id"]

In [5]:
columns = ["business_id", "business_name", "our_score", "stars"]
df = pd.DataFrame(columns=columns)
#usiness.info()

In [6]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15924 entries, 0 to 15923
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   15924 non-null  int64 
 1   review_id    15924 non-null  object
 2   user_id      15924 non-null  object
 3   business_id  15924 non-null  object
 4   stars        15924 non-null  int64 
 5   useful       15924 non-null  int64 
 6   funny        15924 non-null  int64 
 7   cool         15924 non-null  int64 
 8   text         15924 non-null  object
 9   date         15924 non-null  object
dtypes: int64(5), object(5)
memory usage: 1.2+ MB


In [7]:
reviews = reviews.drop(["Unnamed: 0"], axis = 1)

In [8]:
columns_review = ["review_id","user_id","business_id","stars","useful","funny","cool","text","date","sentiment"]
final_reviews = pd.DataFrame(columns= columns_review)


In [9]:
final_reviews

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentiment


In [10]:
for i in range(len(ids)): 
    id_curr = ids.iloc[i].id
    
    df = df.append({"business_id": id_curr}, ignore_index=True)
    
    #Review data that matches the current business id 
    all_reviews = reviews[reviews["business_id"] == id_curr]
    
    #vectorize the reviews for that resturant 
    X_test = cv.transform(all_reviews["text"]).toarray()
    #creates the predicted sentiment for each review 
    y_pred = classifier.predict(X_test)
    #you dont have to round 
    rounded_y_pred = np.mean(y_pred)*100
    #update the column with our calculated score 
    business.loc[business['business_id'] == id_curr, "food_check_score"] = rounded_y_pred
    
    
    #This is for Agile dont regard this part 
    #===========================================
    #add a column for sentiment in all_reviews 
    all_reviews["sentiment"] = y_pred
    
    #split positive and negative 
    only_positive = all_reviews.loc[all_reviews['sentiment'] == 1]
    only_negative = all_reviews.loc[all_reviews['sentiment']==0]
    
    #order by the rankings and take the top 5 of each pos/neg
    only_positive = only_positive.sort_values(by=["stars"],ascending = False).head(5)
    only_negative = only_negative.sort_values(by=["stars"],ascending = True).head(5)
    
    
    #append the data frame to a final one containing all the reviews 
    final_reviews = final_reviews.append(only_positive)
    final_reviews = final_reviews.append(only_negative)
    #pd.concat([final_reviews,only_positive])
    
    


/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edmond/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [11]:
final_reviews.shape



(1000, 10)

In [12]:
from IPython.core.display import HTML

#display(HTML(final_reviews.to_html()))

In [13]:
final_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 5085 to 956
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    1000 non-null   object
 1   user_id      1000 non-null   object
 2   business_id  1000 non-null   object
 3   stars        1000 non-null   object
 4   useful       1000 non-null   object
 5   funny        1000 non-null   object
 6   cool         1000 non-null   object
 7   text         1000 non-null   object
 8   date         1000 non-null   object
 9   sentiment    1000 non-null   object
dtypes: object(10)
memory usage: 85.9+ KB


In [15]:
final_reviews.to_csv("reviews.csv",encoding="utf-8")